## Домашнее задание - 6


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
!pip install implicit

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [4]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [6]:
recommender = MainRecommender(data_train_lvl_1)

In [7]:
recommender.get_als_recommendations(2375, N=200)

[899624,
 871756,
 999714,
 1106523,
 1116376,
 883932,
 12731432,
 1044078,
 1046545,
 1094833,
 1003616,
 1022827,
 913278,
 5569230,
 8090521,
 854852,
 1102207,
 1090931,
 832678,
 12731543,
 841220,
 925862,
 1138467,
 1123146,
 1009770,
 827919,
 896613,
 1051323,
 12262978,
 1131047,
 6534480,
 1134678,
 949965,
 844179,
 828106,
 1081177,
 823990,
 865456,
 9526410,
 963727,
 1081262,
 8090537,
 1000753,
 1026984,
 999779,
 863447,
 1107420,
 828393,
 830304,
 861279,
 1097001,
 1004906,
 963971,
 851819,
 9836106,
 9835223,
 898958,
 1137284,
 947858,
 8091550,
 1132771,
 879504,
 8180870,
 1137346,
 1103691,
 826666,
 823704,
 1092937,
 1012587,
 822178,
 1001702,
 9419769,
 1084613,
 845208,
 870547,
 865528,
 1037863,
 965267,
 9707340,
 944534,
 1131438,
 973181,
 834911,
 868909,
 949151,
 1042544,
 1068719,
 12810391,
 1033046,
 995896,
 847066,
 835300,
 847962,
 1067695,
 1038663,
 847790,
 12262778,
 898068,
 1096573,
 5567143,
 917384,
 938138,
 1056509,
 910032,
 98

In [8]:
recommender.get_own_recommendations(2375, N=200)

[948640,
 918046,
 847962,
 907099,
 873980,
 884694,
 10285454,
 1107760,
 7169090,
 979674,
 10308345,
 1069531,
 974766,
 1015474,
 950935,
 847066,
 1102207,
 1020770,
 9521787,
 974265,
 940996,
 8019845,
 5567194,
 12811490,
 1003616,
 973181,
 890719,
 982955,
 9677152,
 998519,
 1072685,
 1131382,
 1021715,
 12263119,
 960791,
 7441873,
 986021,
 956666,
 1038692,
 9677748,
 9297223,
 927030,
 12757653,
 1046919,
 6391532,
 989069,
 1068451,
 951954,
 835300,
 937343,
 1047249,
 13876348,
 1061732,
 981601,
 1121028,
 1087547,
 828393,
 996269,
 951951,
 1036093,
 1023815,
 5570408,
 827667,
 1082454,
 1006878,
 5570048,
 841309,
 1078652,
 1115553,
 1056492,
 1138467,
 1004945,
 947858,
 1092885,
 1121694,
 938138,
 8019916,
 827919,
 984315,
 10341855,
 883932,
 8291322,
 1096794,
 1028938,
 1087618,
 8020166,
 1082185,
 866871,
 930666,
 825994,
 910151,
 823990,
 848029,
 896613,
 12301839,
 1117219,
 1135258,
 869868,
 1046545,
 899624,
 6442594,
 1137775,
 825343,
 104290

In [9]:
recommender.get_similar_items_recommendation(2375, N=200)

[1046545,
 1044078,
 841220,
 1078652,
 15778319,
 999999,
 1003616,
 896613,
 885863,
 1025535,
 863447,
 878996,
 1127831,
 871611,
 898068,
 866871,
 828106,
 1018740,
 1127831,
 1084613,
 938138,
 907099,
 835300,
 844179,
 899624,
 13876914,
 941883,
 7139529,
 985999,
 1037417,
 9835903,
 1068719,
 1000753,
 992826,
 1131488,
 993912,
 6633210,
 848029,
 1022428,
 5567876,
 14111400,
 916122,
 869322,
 6463877,
 10198511,
 1035143,
 947201,
 1103618,
 8065410,
 1008032,
 856827,
 960732,
 6979526,
 965766,
 15831209,
 1112238,
 844179,
 828106,
 883068,
 1105488,
 925862,
 825343,
 963971,
 1026118,
 899459,
 997987,
 8090537,
 882247,
 5569845,
 948650,
 920654,
 12262778,
 845677,
 1040346,
 957411,
 969945,
 866025,
 5591170,
 1133312,
 828489,
 995896,
 1004906,
 1055503,
 12132277,
 8011291,
 913278,
 9677149,
 1054262,
 885309,
 823704,
 1108296,
 12757425,
 12757653,
 1115187,
 1048507,
 1053754,
 9859111,
 999270,
 1018740,
 7410348,
 1101477,
 885863,
 9707240,
 901916,


In [10]:
recommender.get_similar_users_recommendation(2375, N=200)

[820612,
 896757,
 9553048,
 12427353,
 9392700,
 1012801,
 918638,
 949257,
 8181555,
 974265,
 1124971,
 10457044,
 920025,
 977927,
 6979437,
 985889,
 904241,
 959455,
 995502,
 1102003,
 841365,
 959830,
 839605,
 822970,
 965772,
 1020208,
 9245108,
 12523928,
 1057168,
 9392953,
 1135253,
 9553382,
 5707857,
 902640,
 1117219,
 1057168,
 1023160,
 948239,
 5569135,
 1118120,
 8019902,
 820582,
 1026945,
 927028,
 943940,
 994577,
 1027835,
 873044,
 7443137,
 967041,
 917033,
 857538,
 988277,
 999189,
 1118946,
 847803,
 12487356,
 916990,
 1031316,
 865233,
 873324,
 1071196,
 1072693,
 855454,
 1108168,
 939860,
 5569309,
 951834,
 977927,
 835578,
 972445,
 938165,
 939860,
 1015539,
 9553335,
 983665,
 928263,
 8066803,
 1065538,
 874563,
 5574377,
 1047525,
 1117602,
 854133,
 979674,
 1097398,
 9677454,
 896757,
 951188,
 1055403,
 972191,
 12263857,
 827781,
 1028238,
 1058709,
 1012801,
 1107760,
 6391557,
 1088979,
 9392953,
 945909,
 837495,
 1114811,
 977927,
 875392

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [11]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [12]:
users_cold = np.setdiff1d(result_lvl_1['user_id'], data_train_lvl_1['user_id']).tolist()
users_cold

[296, 1813, 1984]

In [13]:
result_lvl_1 = result_lvl_1[~result_lvl_1['user_id'].isin(users_cold)]

In [14]:
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [15]:
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['own_recommendations'], row['actual'], 200), axis=1).mean()

0.13537278412833242

In [16]:
result_lvl_1['als_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['als_recommendations'], row['actual'], 200), axis=1).mean()

0.10338324934423698

In [17]:
result_lvl_1['similar_items_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['similar_items_recommendation'], row['actual'], 200), axis=1).mean()

0.08899118509838891

In [18]:
result_lvl_1['similar_users_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['similar_users_recommendation'], row['actual'], 200), axis=1).mean()

0.008586453783162366

In [19]:
result_lvl_1.head(2)

,user_id,actual,own_recommendations,als_recommendations,similar_items_recommendation,similar_users_recommendation
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[5577022, 962615, 920200, 1082212, 976214, 856...","[844818, 5582712, 9297615, 5577022, 9670830, 9...","[937110, 977559, 983753, 932949, 939860, 84136..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[865528, 5569230, 1110244, 972665, 1092835, 10...","[8090509, 5569845, 1044078, 985999, 880888, 81...","[882826, 1015539, 898363, 933102, 861990, 8531..."


Определим зависимость recall@k от k для own_recommendations при k = {20, 50, 100, 200, 500}:

In [20]:
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=20))
result_lvl_1.apply(lambda row: recall_at_k(row['own_recommendations'], row['actual'], 20), axis=1).mean()

0.03928427679372909

In [21]:
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1.apply(lambda row: recall_at_k(row['own_recommendations'], row['actual'], 50), axis=1).mean()

0.06525657038145175

In [22]:
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))
result_lvl_1.apply(lambda row: recall_at_k(row['own_recommendations'], row['actual'], 100), axis=1).mean()

0.09604492955885034

In [23]:
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))
result_lvl_1.apply(lambda row: recall_at_k(row['own_recommendations'], row['actual'], 500), axis=1).mean()

0.18205324555508678

Ответы:

A)   Наибольший *recall@k* даёт *own recommendtions+top-popular*;

Б)   По мере увеличения *k* наблюдается увеличение *recall@k*;

С)   Наиболее целесообразным значением *k* является значение равное максимальному числу кандидатов.



### Задание 2.

Обучите модель 2-ого уровня, при этом:
    
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар;
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2;
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [24]:
als_recommendations = result_lvl_1[['user_id', 'als_recommendations']]

In [25]:
valid_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index().rename(columns={'item_id': 'actual'})
valid_lvl_2 = valid_lvl_2.merge(als_recommendations, on='user_id', how='left')

In [26]:
valid_lvl_2[valid_lvl_2.als_recommendations.notna()].\
apply(lambda row: precision_at_k(row['als_recommendations'], row['actual'], k=5), axis=1).mean()

0.10558746736292388